# RNN Moneymaker

In [1]:
import csv
import re
import torch
import time
import numpy as np
import matplotlib.pyplot as plt
import json
import os
from datetime import datetime

import torch.optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

In [2]:
root_path = 'data/'

In [22]:
raw_data = {}
all_files = os.listdir(root_path)

for i, filename in enumerate(all_files):
    
    # Only load 1/N of all stocks
    if i % 3 == 0:
    
        len_stocks = len(all_files)
        print("Loading stock {}/{} ({})   ".format(i + 1, len_stocks, filename), end='\r')
        
        with open(root_path + filename) as f:
            if not filename.startswith('.'):
                data = json.load(f)
                                
                prices = []
                dates = []
                for k, v in data.items():
                    prices.append(np.array([ float(i[1]) for i in v.items() if i[0] != "5. volume" ]))
                    dates.append(k)
                
                # reverse so that data is increasing in time
                prices.reverse()
                dates.reverse()
                raw_data[filename.split('.')[0]] = (prices, dates)

print("")
print("Total number of stocks: " + str(len(raw_data)))

Loading stock 3562/3563 (RDCM.txt)      
Total number of stocks: 1187


In [27]:
X_train, X_val = [], []
y_train, y_val = [], []

train_mean, val_mean = [], []
train_std, val_std = [], []

window_size = 122 # a third of a year

plen = 0
for i, items in enumerate(raw_data.items()):
    print("({}/{})".format(i, len(raw_data.items())), end="\r")
    k, v = items
    prices, _ = v
    if len(prices) < window_size + 1:
        continue
    
    prices = torch.tensor(prices).float()
    
    # used for validating that no data is missing
    plen += prices.shape[0] - window_size - 1
    
    for j in range(prices.shape[0] - window_size - 1):
        window = prices[j:j+window_size+1]     # window from 1st to 122nd and pred at 123rd
        
        mean = torch.mean(window)
        std = torch.std(window)
        
        if std == 0:
            break
        
        norm_window = (window - mean) / std
        norm_x = norm_window[:window_size]
        norm_y = norm_window[window_size][3]
        
        # 90% training to 10% validation
        if i % 9 != 0:
            X_train.append(norm_x.unsqueeze(0))
            y_train.append(norm_y.item())
            train_mean.append(mean)
            train_std.append(std)
        else:
            X_val.append(norm_x.unsqueeze(0))
            y_val.append(norm_y.item())
            val_mean.append(mean)
            val_std.append(std)

print("Converting to torch tensors...")
X_train = torch.cat(X_train)
y_train = torch.tensor(y_train).unsqueeze(1)
X_val = torch.cat(X_val)
y_val = torch.tensor(y_val).unsqueeze(1)

train_mean = torch.tensor(train_mean)
val_mean = torch.tensor(val_mean)
train_std = torch.tensor(train_std)
val_std = torch.tensor(val_std)

# optimize for GPU if exists
if torch.cuda.is_available():
    X_train = X_train.cuda()
    y_train = y_train.cuda()
    X_val = X_val.cuda()
    y_val = y_val.cuda()
    
print("Done")

Converting to torch tensors...
Done


In [28]:
print("X_train shape: \t\t", X_train.shape)
print("X_val shape: \t\t", X_val.shape)
print("y_train shape: \t\t", y_train.shape)
print("y_val shape: \t\t", y_val.shape)
print("")

print("train_mean shape: \t", train_mean.shape)
print("val_mean: \t\t", val_mean.shape)
print("train_std: \t\t", train_std.shape)
print("val_std: \t\t", val_std.shape)
print("")

# # false means that something is wrong
# print(plen == X_train.shape[0] + X_val.shape[0])
# print(plen == y_train.shape[0] + y_val.shape[0])
# print(plen == train_mean.shape[0] + val_mean.shape[0])
# print(plen == train_std.shape[0] + val_mean.shape[0])

X_train shape: 		 torch.Size([2570550, 122, 4])
X_val shape: 		 torch.Size([299507, 122, 4])
y_train shape: 		 torch.Size([2570550, 1])
y_val shape: 		 torch.Size([299507, 1])

train_mean shape: 	 torch.Size([2570550])
val_mean: 		 torch.Size([299507])
train_std: 		 torch.Size([2570550])
val_std: 		 torch.Size([299507])

False
False
False
False


In [29]:
# a = torch.sum(torch.sum(X_train.cpu() == X_train.cpu(), dim=1), dim=1) != 0
# print(a.shape)
# b = (y_train.cpu() == y_train.cpu()) != 0

# c = a if torch.sum(a) < torch.sum(b) else b

# X = X[c]
# y = y[c]

# print("new:")
# print(X.shape)
# print(y.shape)

torch.Size([2570550])


### Training a RNN

In [30]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        return self.X[index], self.y[index]

In [31]:
batch_size = 32
dataset = Dataset(X_train, y_train)
loader = DataLoader(dataset, batch_size)

In [32]:
class RNNClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(RNNClassifier, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        
        self.rnn = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.drop = nn.Dropout(0.5)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x, h=None):
        if type(h) == type(None):
            out, hn = self.rnn(x)
        else:
            out, hn = self.rnn(x, h.detach())
        out = self.drop(out)
        out = self.fc(out[:, -1, :])
        return out

In [33]:
input_dim = 4
hidden_dim = 20
output_dim = 1

In [34]:
model = RNNClassifier(input_dim, hidden_dim, output_dim)
if torch.cuda.is_available():
    model = model.to("cuda")
    
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

In [35]:
train_accs = []
val_accs = []
train_losses = []
val_losses = []
epoch = 0

### Load a pre-existing model for further training or prediction

In [ ]:
model = RNNClassifier(input_dim, hidden_dim, output_dim)
if torch.cuda.is_available():
    model = model.to("cuda")
    
optimizer = torch.optim.Adam(model.parameters())

checkpoint = torch.load('assets/partial_model.pt')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

model.train()

### Train the model

In [ ]:
t0 = time.time()
num_epochs = 3
for ep in range(num_epochs):
    tstart = time.time()
    for i, data in enumerate(loader):
        model.train()
        print("{}/{}".format(i, X_train.shape[0]), end='\r')
        optimizer.zero_grad()
        outputs = model(data[0])
        loss = criterion(outputs, data[1])
        loss.backward()
        optimizer.step()
    
        if i % 1000 == 0:
            with torch.no_grad():
                model.eval()
                train_losses.append(loss.item())
                pXval = model(X_val)
                vloss = criterion(pXval, y_val)
                val_losses.append(vloss.item())
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'loss': loss,
                }, 'assets/partial_model.pt')

                print("training loss: {:<3.3f} \t val loss: {:<3.3f}".format(loss, vloss))

    with torch.no_grad():
        model.eval()
        pXval = model(X_val)
        vloss = criterion(pXval, y_val)
        val_losses.append(vloss.item())
        epoch += 1    
        tend = time.time()
        print('epoch: {:<3d} \t time: {:<3.2f} \t val loss: {:<3.3f}'.format(epoch, 
                tend - tstart, vloss.item()))
time_total = time.time() - t0
print('Total time: {:4.3f}, average time per epoch: {:4.3f}'.format(time_total, time_total / num_epochs))

training loss: 0.910 	 val loss: 2.022
training loss: 0.075 	 val loss: 0.194
training loss: 0.212 	 val loss: 0.170
training loss: 0.143 	 val loss: 0.166
training loss: 0.354 	 val loss: 0.167
training loss: 0.135 	 val loss: 0.153
training loss: 0.011 	 val loss: 0.156
training loss: 0.135 	 val loss: 0.178
training loss: 0.497 	 val loss: 0.154
training loss: 0.033 	 val loss: 0.151
training loss: 0.048 	 val loss: 0.155
training loss: 0.236 	 val loss: 0.149
training loss: 0.054 	 val loss: 0.153
training loss: 0.108 	 val loss: 0.154
training loss: 0.386 	 val loss: 0.153
training loss: 0.222 	 val loss: 0.167
training loss: 0.248 	 val loss: 0.153


In [ ]:
# ignore the UserWarning
torch.save(model, 'assets/model.pt')

### Model evaluation

In [ ]:
model = torch.load('assets/model.pt')

In [ ]:
t_losses = [i for i in train_losses if i < 4000]
plt.plot(t_losses)
plt.plot(val_losses)
plt.title('loss history')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.yscale('log')
plt.legend(['train', 'val'])

In [ ]:
model.eval()
pred = model(X_val)
print(criterion(pred, y_val))

### Model predictions compared to standard deviations

In [ ]:
val_batch_size = 1000
val_set_size = X_val.shape[0]
preds = []
with torch.no_grad():
    for i in range(0, val_set_size, val_batch_size):
        start = i
        end = min(i+val_batch_size, val_set_size)
        print("Predicting {} - {}...".format(start, end), end='\r')
        preds.append(model(X_val[start:end]))
pred = torch.cat(preds, dim=0).cpu()
print("")
plt.plot((pred - y_val.cpu()).detach()[1000:1100])
plt.title('std difference')

### Model predictions compared to actual price

In [ ]:
pred_abs = pred * X_std[val_selector][:,:,3] + X_mean[val_selector][:,:,3]
y_val_abs = y_val * X_std[val_selector][:,:,3] + X_mean[val_selector][:,:,3]

plt.plot((pred_abs - y_val_abs).detach()[1000:1100])
plt.title('absolute price difference')

### Testing and visualizing random stock in test data

In [ ]:
val_names_and_dates = np.array(name_and_date)[val_selector]

stock = "AUTO";
stock_selector = torch.tensor(val_names_and_dates[:,0] == (stock + ".txt"))

s_pred_abs = pred_abs[stock_selector]
s_y_val_abs = y_val_abs[stock_selector]

stock_dates = val_names_and_dates[stock_selector][:,1]

In [ ]:
stock_dates.sort()
fig, ax = plt.subplots()
ax.plot(stock_dates[10:20], s_pred_abs.detach()[10:20])
ax.plot(stock_dates[10:20], s_y_val_abs.detach()[10:20])
fig.autofmt_xdate()
start, end = ax.get_xlim()
plt.show()

### Testing random unseen stock (skipped over in the data loading stage)

In [ ]:
seq = []
test_y = []
test_name_and_date = []
window_size = 122

with open(root_path + "AAPL.txt") as f:
    data = json.load(f)

    temp1 = [] # for prices
    temp2 = [] # for name and date

    for k, v in data.items():
        temp1.append(torch.tensor([ float(i[1]) for i in v.items() if i[0] != "5. volume" ]).unsqueeze(0))
        temp2.append(k)

    # reverse so that data is increasing in time
    temp1.reverse()
    temp2.reverse()

    prices = torch.cat(temp1, 0)

    for i in range(len(prices) - window_size - 1):
        seq.append(prices[i:i+window_size].unsqueeze(0))
        test_y.append(prices[i+window_size][3].item()) # predict the closing price
        test_name_and_date.append(['AAPL', temp2[i + window_size + 1]])
        
test_X = torch.cat(seq, 0)
test_y = torch.tensor(test_y).unsqueeze(1) # from (N,) to (N,1)

test_X_mean = torch.mean(test_X, dim=1).unsqueeze(1)
test_X_std = torch.std(test_X, dim=1).unsqueeze(1)

test_X = (test_X - test_X_mean) / test_X_std
test_y = (test_y - test_X_mean[:,:,3]) / test_X_std[:,:,3]

test_pred = model(test_X)
print(criterion(test_pred, test_y))

test_pred = test_pred * test_X_std[:,:,3] + test_X_mean[:,:,3]
test_y = test_y * test_X_std[:,:,3] + test_X_mean[:,:,3]

plt.plot(test_pred.detach()[3000:3100])
plt.plot(test_y[3000:3100])

In [ ]:
dates = [i[1] for i in test_name_and_date]
print(dates)
temp_pred = [ i.data[0] for i in test_pred.detach().numpy() ]
temp_y = [ i.data[0] for i in test_y.detach().numpy() ]

In [ ]:
import matplotlib.ticker as mticker

fig, ax = plt.subplots()
ax.plot(dates[1000:1010], temp_pred[1000:1010])
fig.autofmt_xdate()
start, end = ax.get_xlim()
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.plot(dates[1000:1010], temp_y[1000:1010])
fig.autofmt_xdate()
start, end = ax.get_xlim()
plt.show()